# 원클릭 제출 생성

최신 학습 스크립트를 실행하고 즉시 제출 CSV를 준비합니다.

- 아래 설정 셀만 수정한 뒤 순서대로 실행하세요.
- 학습 스크립트 내부에서 저장 직전 `du -sh / 2>/dev/null`을 자동 실행합니다.

In [1]:
from pathlib import Path
from typing import List
import shlex

# 학습 및 제출 생성 파라미터 설정
raw_train = 'data/train.parquet'
train_path = 'data/processed/train_downsample_1_2.parquet'
ensure_downsample = False
negative_multiplier = 2.0
seed = 42
enable_wandb = False
notes = ''
inference_input = 'data/processed/test.parquet'
submission_path = None  # 예: 'submission/custom_preds.csv'

project_root = Path.cwd().resolve()
train_candidate = project_root / 'src' / 'train' / 'train.py'
if not train_candidate.exists():
    alt_candidate = project_root.parent / 'src' / 'train' / 'train.py'
    if alt_candidate.exists():
        project_root = project_root.parent
        train_candidate = alt_candidate

if not train_candidate.exists():
    raise FileNotFoundError(f"훈련 스크립트를 찾을 수 없습니다: {train_candidate}")

def resolve_path(base: Path, value: str) -> Path:
    path = Path(value)
    return path if path.is_absolute() else (base / path).resolve()

raw_train_path = resolve_path(project_root, raw_train)
train_path_resolved = resolve_path(project_root, train_path)
inference_input_path = resolve_path(project_root, inference_input)
config_dir = (project_root / "model").resolve()
model_dir = config_dir
submission_output = None
if submission_path:
    submission_output = resolve_path(project_root, submission_path)

base_command: List[str] = [
    'python',
    str(train_candidate),
    f'--raw-train={raw_train_path}',
    f'--train-path={train_path_resolved}',
    f'--negative-multiplier={negative_multiplier}',
    f'--seed={seed}',
    f'--config-dir={config_dir}',
    f'--model-dir={model_dir}',
    f'--notes={notes}',
    '--run-inference',
    f'--inference-input={inference_input_path}',
]
if ensure_downsample:
    base_command.append('--ensure-downsample')
if enable_wandb:
    base_command.append('--wandb')
if submission_output:
    base_command.append(f'--submission-path={submission_output}')

print(f'프로젝트 루트: {project_root}')
print('실행 커맨드:')
print(' '.join(shlex.quote(part) for part in base_command))


FileNotFoundError: 훈련 스크립트를 찾을 수 없습니다: /Competition/toss-ad-click-prediction/notebooks/src/train/train.py

In [ ]:
import subprocess

result = subprocess.run(base_command, check=False, cwd=project_root)
if result.returncode != 0:
    raise RuntimeError(f'명령 실행 실패 (exit code {result.returncode})')
